### 0.0 Imports

In [646]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from collections import Counter, OrderedDict

# Load library
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# download the set of stop words the first time
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')

import re
import ast
import math

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

%env GOOGLE_APPLICATION_CREDENTIALS=/Users/cesleemontgomery/masters/cs6460/CourseRec/courserec-adfe3-firebase-adminsdk-pm32q-6cd7a11195.json

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/cesleemontgomery/masters/cs6460/CourseRec/courserec-adfe3-firebase-adminsdk-pm32q-6cd7a11195.json


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cesleemontgomery/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/cesleemontgomery/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/cesleemontgomery/nltk_data...
[nltk_data]   Package words is already up-to-date!


### 0.1 Extract Data from Corpuses: Courses - Video Subtitles (Udacity), User Interests - Academic-related Articles (Wikipedia)

In [636]:
course_subs_DF = pd.read_csv('../../data/processed/course_data_udacity.csv')
# course_subs_DF = course_subs_DF[pd.notnull(course_subs_DF.course_name)] #we found all the courses when looking at udacity course pages
wiki_page_path_DF = pd.read_csv('../../data/processed/academic_outline_wikipedia_pages.csv')

### 0.2 Utils

In [771]:
words = set(nltk.corpus.words.words())

# Building function for easy comparison
def get_counter_from_list(templist, words=words):

    #lowercase,remove punctuation and non-alpha, split whitespace
    templist = " ".join(w for w in nltk.wordpunct_tokenize(str(templist)) \
         if w.lower() in words).lower()
    templist = re.sub("[^a-zA-Z\s]+", "", templist).split()

    # Load stop words, lemmatizer
    stop_words = stopwords.words('english')
    lemma = nltk.wordnet.WordNetLemmatizer()

    # Remove stop words, lowercase, lemmatize
    counter = Counter([lemma.lemmatize(word.lower()) for word in templist if word not in stop_words])
    return counter

# 2.3.v2- Modeling - content_based_model - Improving Accuracy of Mooney et al. text categorization ML approach.

### Objective: In the end we want to improve the accuracy of the recommendations as judged by this table.

To do this more blue tiles should be towards the top.  Signifying 'ground truth' courses are ranking high for their given category.

### Approach:

1. To do this we will start by analyzing the input keywords and eventually the matched keywords to see why recommendations are somewhat poor.

![Offline Evaluation - CS Interests](../../img/v1_offlineeval_full.png)

### 1. Visual Inspection of Keywords for Software Engineering

To begin, I will visually inspect the match between Software Engineering (**interest**) and Software Development Process (**course**).  This is convenient for me since I took this class last semester.  Although its not the worse match as it does show up top 10, it would still be worth investigating and then moving on to worse match courses.

In [928]:
# tempdf = "".join(course_subs_DF[course_subs_DF['course_slug'] == 'Machine Learning'].subtitles.values).lower()
" ".join(w for w in nltk.wordpunct_tokenize(str(course_subs_DF[course_subs_DF['course_slug'] == 'Advanced Operating Systems'].subtitles.values)) \
         if w.lower() in words)[0:10000]

'The right answer is That is in be forged theres no way to the protection mechanism that is built into language So if I have a data structure defined In And if nI have pointer to the data structure the only way you can use that pointer as a pointer to that type of data structure You cannot take a data structure cast it to appear like something else this is something that we as C maybe very used to doing but not something that is possible in that is what us to implement protection in and to as by the is a concrete example of an Its a fairly standard implementation say operating system but it is an extension on top of the spin is a more fun example A client application that is directly on top spin as an extension In other is no operating system A display client an extension interface to implement the for video that is going to sent by a video server So both of these are on top of basic and provide exactly the functionality that for the video server application And bounding box here is sh

In [630]:
# tempdf = "".join(course_subs_DF[course_subs_DF['course_slug'] == 'Machine Learning'].subtitles.values).lower()
" ".join(w for w in nltk.wordpunct_tokenize(str(course_subs_DF[course_subs_DF['course_slug'] == 'Software Development Process'].subtitles.values)) \
         if w.lower() in words)[0:1000]

'The second question I want to ask you is which model is the most suitable if you Would you rather use a waterfall model a spiral model or evolutionary Now before we jump into the next I just want to take a very quick and quiz just to make sure that you attention to what I just So nI want to ask you what are the traditional phases engineering abstraction implementation verification and Or maybe design implementation verification and validation maintenance Or design implementation verification and and next type of process that I to discuss are Agile Development And this a group of development that are on highly iterative and incremental development And in particular nIm going to discuss Test Driven Development or The on the iteration of three main phases In first one that we mark as red we test that encode our and for which havent written code yet And therefore they will fail obviously were in this sort of red or fail phase From phase we move to this phase in which after we the just eno

In [637]:
tempdf = course_subs_DF[course_subs_DF['course_slug'] == 'Software Development Process'].subtitles.values

print("Most Common Words in 'Software Development Process' course: \n")
for i in get_counter_from_list(tempdf, words).most_common(200): print(i)

Most Common Words in 'Software Development Process' course: 

('one', 549)
('system', 530)
('going', 456)
('case', 397)
('want', 381)
('see', 364)
('also', 357)
('code', 353)
('class', 352)
('use', 350)
('test', 345)
('n', 316)
('example', 300)
('first', 266)
('way', 247)
('two', 240)
('might', 220)
('testing', 220)
('na', 206)
('look', 199)
('need', 186)
('right', 185)
('go', 182)
('different', 176)
('phase', 170)
('well', 168)
('point', 165)
('know', 161)
('make', 156)
('time', 155)
('like', 154)
('many', 153)
('coverage', 153)
('git', 151)
('file', 147)
('architecture', 145)
('something', 143)
('project', 142)
('design', 141)
('program', 140)
('process', 138)
('get', 136)
('course', 135)
('kind', 134)
('really', 132)
('method', 129)
('actually', 123)
('development', 122)
('important', 121)
('second', 120)
('set', 119)
('basically', 119)
('fact', 118)
('repository', 118)
('dont', 117)
('therefore', 115)
('start', 115)
('used', 113)
('branch', 111)
('thats', 110)
('main', 107)
('numbe

I can definitely see some keywords among the bunch such as *requirements*, *architecture*, and *engineering*.

But this feels lacking.  I fear it would both mostly match on bogus keywords or when I go to show these as explanations, they will actually reduce user trust in the tool.

#### 1.1 This points me in two new directions:

#### A. Using Course Syllabi as the basis of course keywords will make explanations more trust-worthy.
These keywords have specifically been chosen by course instructors as representative of the course, so there's less worry about random words showing up.


<img src="../../img/SDP_Syllabus.png" alt="Drawing" style="width: 750px; height: 350px;"/>

#### B. Possibly mixing in the subtitles-based course keyword model for calculation of matches for breadth.
For example, *"engineering"* may not be a descriptive keyword for a user explanation but it does give us useful signals about this course vs an Algorithms course for example.

### 2.1 Use Course Syllabi as the basis of course keywords

#### 2.1.1 Let's visually compare matches for subtitles vs syllabus

In [720]:
tempstr_subtitles = course_subs_DF[course_subs_DF['course_slug'] == 'Software Development Process'].subtitles.values
templist_subtitles = list(get_counter_from_list(tempstr_subtitles, words).keys())

print("Most Common Words in 'Software Development Process' course: \n")
for i in get_counter_from_list(tempstr_subtitles, words).most_common(200): print(i)

Most Common Words in 'Software Development Process' course: 

('one', 549)
('system', 530)
('going', 456)
('case', 397)
('want', 381)
('see', 364)
('also', 357)
('code', 353)
('class', 352)
('use', 350)
('test', 345)
('n', 316)
('example', 300)
('first', 266)
('way', 247)
('two', 240)
('might', 220)
('testing', 220)
('na', 206)
('look', 199)
('need', 186)
('right', 185)
('go', 182)
('different', 176)
('phase', 170)
('well', 168)
('point', 165)
('know', 161)
('make', 156)
('time', 155)
('like', 154)
('many', 153)
('coverage', 153)
('git', 151)
('file', 147)
('architecture', 145)
('something', 143)
('project', 142)
('design', 141)
('program', 140)
('process', 138)
('get', 136)
('course', 135)
('kind', 134)
('really', 132)
('method', 129)
('actually', 123)
('development', 122)
('important', 121)
('second', 120)
('set', 119)
('basically', 119)
('fact', 118)
('repository', 118)
('dont', 117)
('therefore', 115)
('start', 115)
('used', 113)
('branch', 111)
('thats', 110)
('main', 107)
('numbe

In [713]:
tempdict = ast.literal_eval(course_subs_DF[course_subs_DF['course_slug'] == 'Software Development Process'].course_syllabus_dict.values[0])
#strategy 1 = bag of words: how many of words in bag found?
templist_syllabus = []
for k, v in tempdict.items():
    templist_syllabus.extend(k.split())
    [templist_syllabus.extend(w for w in words.split() if w not in stopwords.words('english')) for words in v]

In [721]:
#find common words in article and syllabus vs subtitles

print("Most Common Words in 'Software engineering' course: \n")
    
tempstr_wikiarticle = wiki_page_path_DF[wiki_page_path_DF.page_href == '/wiki/Software_engineering'].page_text.values
templist_wikiarticle = list(get_counter_from_list(tempstr_wikiarticle, words).keys())
for i in get_counter_from_list(tempstr_wikiarticle, words).most_common(200): print(i)

Most Common Words in 'Software engineering' course: 

('engineering', 87)
('n', 22)
('professional', 22)
('computer', 21)
('development', 19)
('certification', 14)
('science', 11)
('process', 11)
('knowledge', 10)
('work', 10)
('also', 10)
('information', 10)
('engineer', 10)
('many', 8)
('use', 7)
('term', 7)
('body', 7)
('world', 7)
('application', 6)
('approach', 6)
('u', 6)
('management', 6)
('level', 6)
('education', 6)
('systematic', 5)
('practice', 5)
('program', 5)
('defined', 5)
('society', 5)
('field', 5)
('experience', 4)
('design', 4)
('testing', 4)
('discipline', 4)
('real', 4)
('used', 4)
('architecture', 4)
('number', 4)
('institute', 4)
('research', 4)
('configuration', 4)
('related', 4)
('profession', 4)
('curriculum', 4)
('technology', 4)
('p', 4)
('due', 4)
('percent', 4)
('distance', 4)
('human', 4)
('include', 3)
('statistic', 3)
('maintenance', 3)
('formally', 3)
('sub', 3)
('specific', 3)
('one', 3)
('early', 3)
('deal', 3)
('business', 3)
('key', 3)
('various', 

Pull out 1000 most common words

In [755]:
#pull out top 1000 most common wiki article words
counter = Counter()
for idx, row in wiki_page_path_DF.iterrows():
    print(row['page_href'])
    if pd.notnull(row['page_text']):
        counter.update([w.lower() for w in row['page_text'].split()])

/wiki/Humanities
/wiki/The_arts
/wiki/Performing_arts
nan
/wiki/Accompaniment
/wiki/Chamber_music
/wiki/Church_music
/wiki/Conducting
/wiki/Early_music
nan
/wiki/Musical_composition
/wiki/Music_education
/wiki/Music_history
/wiki/Musicology
/wiki/Ethnomusicology
/wiki/Music_theory
/wiki/Orchestra
/wiki/Organology
/wiki/Sound_recording_and_reproduction
nan
/wiki/Choreography
/wiki/Dance_notation
/wiki/Ethnochoreology
/wiki/History_of_dance
nan
/wiki/Television_studies
nan
/wiki/Acting
/wiki/Theatre_director
/wiki/Dramaturgy
/wiki/History_of_theater
/wiki/Musical_theatre
/wiki/Playwrighting
/wiki/Puppetry
/wiki/Scenography
/wiki/Scenic_design
/wiki/Ventriloquism
nan
/wiki/Animation
/wiki/Film_criticism
/wiki/Filmmaking
/wiki/Film_theory
/wiki/Live_action
/wiki/Visual_arts
/wiki/Fine_arts
/wiki/Graphic_arts
nan
/wiki/Applied_arts
/wiki/Animation
/wiki/Calligraphy
/wiki/Decorative_arts
/wiki/Mixed_media
/wiki/Printmaking
/wiki/Studio_art
nan
/wiki/Design
/wiki/History
/wiki/African_history

In [769]:
stop_words_wiki = list(dict(counter.most_common(1000)).keys())

Subtitle keywords:

In [770]:
set(word for word in templist_subtitles if (word in templist_wikiarticle) and (word not in stop_words_wiki))

{'abstraction',
 'acceptable',
 'accepted',
 'actually',
 'add',
 'added',
 'affected',
 'agile',
 'appear',
 'assessment',
 'associate',
 'assume',
 'average',
 'avoid',
 'background',
 'becoming',
 'big',
 'carefully',
 'carried',
 'cloud',
 'collected',
 'combination',
 'communicate',
 'component',
 'conference',
 'configuration',
 'controversial',
 'core',
 'cost',
 'creation',
 'creative',
 'crisis',
 'currently',
 'curriculum',
 'cycle',
 'day',
 'deal',
 'death',
 'decided',
 'demand',
 'despite',
 'detailed',
 'discover',
 'distributed',
 'documentation',
 'efficiently',
 'elicitation',
 'eligible',
 'emphasis',
 'employ',
 'engineer',
 'enough',
 'ensure',
 'enter',
 'entry',
 'equivalent',
 'ever',
 'extensive',
 'faster',
 'fill',
 'flexible',
 'follow',
 'formally',
 'framework',
 'front',
 'fully',
 'glossary',
 'goal',
 'grow',
 'guide',
 'hall',
 'hand',
 'hardware',
 'hence',
 'hill',
 'hold',
 'however',
 'identification',
 'implementation',
 'improvement',
 'industry'

Syllabus Keywords:

In [768]:
set(word.lower() for word in templist_syllabus if word.lower() in templist_wikiarticle)

{'agile',
 'analysis',
 'approach',
 'architecture',
 'based',
 'change',
 'component',
 'configuration',
 'construction',
 'continuous',
 'cost',
 'creation',
 'crisis',
 'cycle',
 'data',
 'definition',
 'design',
 'development',
 'discipline',
 'engineering',
 'environment',
 'first',
 'general',
 'high',
 'integration',
 'level',
 'life',
 'main',
 'maintenance',
 'method',
 'model',
 'negative',
 'object',
 'perspective',
 'process',
 'project',
 'run',
 'system',
 'systematic',
 'test',
 'testing',
 'two',
 'use',
 'verification',
 'version'}

From this we can see that the syllabus does seem to contain the highest number of useful keywords, while minimizing random words.  This points me towards continuing with using syllabus keywords to drive the model.  I still would like to consider bigrams (especially for evaluation labels).

### 2.2 Let's compute a mutivariate Naive Bayes model of Syllabus keywords

TODO - Consider bigrams, especially for explanation

### 4. v2 Multinomial Naive Bayes implementation - remove top 1000 most common wiki articles words, to normalize uniformally limit # model keywords

#### 4.1 Build table of P( word | Course ) words in Vocabulary x Courses*

*\*prior calculation not required since it won't affect rank of results.*

In [834]:
#pull out top 1000 most common wiki article words
counter_wiki = Counter()
for idx, row in wiki_page_path_DF.iterrows():
    print(row['page_href'])
    if pd.notnull(row['page_text']):
        counter_wiki.update([w.lower() for w in row['page_text'].split()])

/wiki/Humanities
/wiki/The_arts
/wiki/Performing_arts
nan
/wiki/Accompaniment
/wiki/Chamber_music
/wiki/Church_music
/wiki/Conducting
/wiki/Early_music
nan
/wiki/Musical_composition
/wiki/Music_education
/wiki/Music_history
/wiki/Musicology
/wiki/Ethnomusicology
/wiki/Music_theory
/wiki/Orchestra
/wiki/Organology
/wiki/Sound_recording_and_reproduction
nan
/wiki/Choreography
/wiki/Dance_notation
/wiki/Ethnochoreology
/wiki/History_of_dance
nan
/wiki/Television_studies
nan
/wiki/Acting
/wiki/Theatre_director
/wiki/Dramaturgy
/wiki/History_of_theater
/wiki/Musical_theatre
/wiki/Playwrighting
/wiki/Puppetry
/wiki/Scenography
/wiki/Scenic_design
/wiki/Ventriloquism
nan
/wiki/Animation
/wiki/Film_criticism
/wiki/Filmmaking
/wiki/Film_theory
/wiki/Live_action
/wiki/Visual_arts
/wiki/Fine_arts
/wiki/Graphic_arts
nan
/wiki/Applied_arts
/wiki/Animation
/wiki/Calligraphy
/wiki/Decorative_arts
/wiki/Mixed_media
/wiki/Printmaking
/wiki/Studio_art
nan
/wiki/Design
/wiki/History
/wiki/African_history

In [825]:
#pull out top 10000 most common subtitle words
counter_subtitles = Counter()
for idx, row in course_subs_DF.iterrows():
    print(row['course_name'])
    if pd.notnull(row['subtitles']):
        counter_subtitles.update([w.lower() for w in row['subtitles'].split()])

Artificial Intelligence for Robotics
Intro to High-Performance Computing
Machine Learning
Database Systems Concepts and Design
Knowledge-Based Artificial Intelligence: Cognitive Systems
Educational Technology
Introduction to Operating Systems
Computer Networks
Special Topics: Big Data for Health Informatics
Advanced Operating Systems
Compilers - Theory and Practice
Software Analysis
Data and Visual Analytics
Computer Vision
High Performance Computer Architecture
Network Security
Artificial Intelligence
Intro to Health Informatics
Introduction to Information Security
Embedded Systems Optimization
Computational Photography
Machine Learning for Trading
Reinforcement Learning
Human-Computer Interaction
Software Development Process
Intro to Cyber Physical Systems Security
Intro to Graduate Algorithms
Software Architecture and Design


In [828]:
#pull out top 10000 most common subtitle words
counter_overrepresented = Counter()
for idx, row in course_subs_DF[course_subs_DF.course_name.isin(['Machine Learning', 'Reinforcement Learning', 'Computer Vision', 'Artificial Intelligence'])].iterrows():
    print(row['course_name'])
    if pd.notnull(row['subtitles']):
        counter_overrepresented.update([w.lower() for w in row['subtitles'].split()])

Machine Learning
Computer Vision
Artificial Intelligence
Reinforcement Learning


In [831]:
counter_overrepresented.most_common(1000)

[('the', 55494),
 ('gtgt', 38204),
 ('a', 37724),
 ('to', 34798),
 ('and', 30716),
 ('that', 30336),
 ('of', 29217),
 ('is', 23713),
 ('so', 21631),
 ('you', 21039),
 ('we', 18270),
 ('i', 17874),
 ('in', 16267),
 ('it', 15102),
 ('this', 14615),
 ('que', 10200),
 ('do', 9533),
 ('what', 8981),
 ('right', 8974),
 ('de', 8642),
 ('e', 8016),
 ('going', 7994),
 ('its', 7843),
 ('have', 7522),
 ('for', 7276),
 ('if', 7260),
 ('but', 7184),
 ('be', 7183),
 ('o', 6829),
 ('just', 6652),
 ('one', 6577),
 ('no', 6500),
 ('can', 6416),
 ('thats', 6306),
 ('as', 6255),
 ('are', 6239),
 ('were', 6118),
 ('all', 6115),
 ('about', 6076),
 ('well', 5145),
 ('or', 5134),
 ('okay', 4966),
 ('like', 4925),
 ('then', 4869),
 ('here', 4854),
 ('at', 4847),
 ('with', 4494),
 ('not', 4404),
 ('know', 4222),
 ('on', 4129),
 ('some', 4002),
 ('get', 3911),
 ('b', 3907),
 ('actually', 3842),
 ('x', 3841),
 ('because', 3647),
 ('which', 3549),
 ('um', 3546),
 ('by', 3521),
 ('would', 3491),
 ('voc', 3485),
 (

In [832]:
counter_subtitles.most_common(1000)

[('the', 235784),
 ('to', 122692),
 ('and', 109607),
 ('a', 107778),
 ('of', 106464),
 ('is', 93759),
 ('that', 92709),
 ('we', 69914),
 ('in', 64756),
 ('this', 61299),
 ('so', 55497),
 ('you', 46254),
 ('it', 44163),
 ('gtgt', 38284),
 ('for', 33109),
 ('i', 31381),
 ('are', 29360),
 ('be', 28599),
 ('have', 27751),
 ('can', 26400),
 ('what', 23563),
 ('as', 22524),
 ('one', 21831),
 ('do', 21488),
 ('if', 20901),
 ('on', 20520),
 ('going', 19161),
 ('at', 19123),
 ('or', 18953),
 ('here', 18665),
 ('its', 18275),
 ('with', 17719),
 ('but', 16600),
 ('which', 16535),
 ('then', 16404),
 ('an', 15348),
 ('all', 15340),
 ('these', 14799),
 ('not', 14691),
 ('about', 14218),
 ('now', 14197),
 ('by', 14050),
 ('were', 13836),
 ('will', 13596),
 ('from', 13448),
 ('just', 13397),
 ('que', 12673),
 ('thats', 12475),
 ('right', 12066),
 ('de', 12042),
 ('how', 11661),
 ('well', 11566),
 ('there', 11265),
 ('because', 11134),
 ('some', 11024),
 ('like', 10411),
 ('two', 10320),
 ('no', 10072)

In [836]:
counter_wiki.most_common(200)

[('the', 229952),
 ('of', 169134),
 ('and', 127118),
 ('in', 87998),
 ('to', 81534),
 ('a', 76618),
 ('is', 48180),
 ('as', 38048),
 ('that', 35100),
 ('for', 29464),
 ('by', 26263),
 ('are', 24889),
 ('with', 22555),
 ('or', 22437),
 ('on', 20007),
 ('be', 18934),
 ('an', 16901),
 ('from', 15714),
 ('was', 15224),
 ('which', 14677),
 ('it', 13872),
 ('this', 13867),
 ('such', 10814),
 ('have', 10765),
 ('can', 10534),
 ('not', 9929),
 ('their', 9626),
 ('also', 9519),
 ('at', 9192),
 ('has', 9188),
 ('other', 8337),
 ('social', 8334),
 ('were', 7816),
 ('more', 7521),
 ('they', 7294),
 ('these', 7063),
 ('one', 7030),
 ('–', 6964),
 ('some', 6604),
 ('its', 6564),
 ('but', 6545),
 ('theory', 6311),
 ('many', 6193),
 ('may', 6157),
 ('been', 5826),
 ('used', 5635),
 ('between', 5587),
 ('new', 5497),
 ('all', 5465),
 ('his', 5411),
 ('there', 5306),
 ('study', 5233),
 ('into', 5180),
 ('most', 5047),
 ('first', 4999),
 ('who', 4523),
 ('than', 4490),
 ('history', 4299),
 ('he', 4155),


In [901]:
stopwords_from_data = set(stopwords.words('english'))
for counter_tuple in [counter_overrepresented.most_common(1000), counter_subtitles.most_common(1000), counter_wiki.most_common(200)]:
    stopwords_from_data.update(list(dict(counter_tuple).keys()))

In [902]:
len(stopwords_from_data)

1409

In [906]:
words = set(nltk.corpus.words.words())



#create vocab from course subtitles
vocabulary = set()

work_df = course_subs_DF.copy()
work_df['subtitles_word_freq'] = pd.Series(dtype=object)

#1. Calc word frequencies for each course
for idx, course in work_df.iterrows():
    print(course.course_name)
    #add Counter to df
    word_freqs = get_counter_from_list(course.subtitles, words)
    print(word_freqs.most_common(10), '\n\n')
    
    vocabulary.update(list(word_freqs.keys()))
    work_df.at[idx,'subtitles_word_freq'] = word_freqs
    
#calc likelihoods for each word | course
NB_model_features_DF = pd.DataFrame(index=sorted(vocabulary), columns=course_subs_DF.course_name)

#2. log likelihood = log of rel. word frequency
    # count of occurences + 1 / total words + Vocab (laplace smoothing)
for idx, course in work_df.iterrows():
    word_freqs = course.subtitles_word_freq
    for word, count in word_freqs.items():
        logLikelihood = math.log(count + 1 / float(sum(word_freqs.values()) + len(vocabulary)))
        NB_model_features_DF.at[word, course.course_name] = logLikelihood

Artificial Intelligence for Robotics
[('de', 2895), ('la', 2231), ('e', 1711), ('x', 1243), ('el', 1079), ('en', 850), ('robot', 787), ('un', 678), ('se', 656), ('para', 594)] 


Intro to High-Performance Computing
[('one', 601), ('time', 523), ('n', 470), ('algorithm', 375), ('two', 374), ('p', 302), ('want', 282), ('first', 268), ('vertex', 265), ('memory', 262)] 


Machine Learning
[('de', 6329), ('e', 4809), ('um', 2830), ('right', 2600), ('going', 2438), ('one', 2274), ('x', 2161), ('se', 2113), ('thats', 2005), ('para', 1941)] 


Database Systems Concepts and Design
[('user', 599), ('one', 445), ('look', 277), ('data', 267), ('going', 217), ('table', 214), ('regular', 213), ('first', 210), ('two', 207), ('result', 204)] 


Knowledge-Based Artificial Intelligence: Cognitive Systems
[('one', 660), ('knowledge', 660), ('problem', 655), ('example', 642), ('well', 501), ('u', 494), ('case', 441), ('agent', 441), ('reasoning', 424), ('learning', 404)] 


Educational Technology
[('reall

In [907]:
NB_model_features_DF.tail(20)

course_name,Artificial Intelligence for Robotics,Intro to High-Performance Computing,Machine Learning,Database Systems Concepts and Design,Knowledge-Based Artificial Intelligence: Cognitive Systems,Educational Technology,Introduction to Operating Systems,Computer Networks,Special Topics: Big Data for Health Informatics,Advanced Operating Systems,...,Introduction to Information Security,Embedded Systems Optimization,Computational Photography,Machine Learning for Trading,Reinforcement Learning,Human-Computer Interaction,Software Development Process,Intro to Cyber Physical Systems Security,Intro to Graduate Algorithms,Software Architecture and Design
za,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zag,NaN,NaN,NaN,NaN,1.49024e-05,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zaman,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zebra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.693152,...,NaN,NaN,1.79176,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zero,4.26268,4.11087,6.44095,1.09862,1.3863,NaN,3.80666,3.4012,1.79176,4.43082,...,4.17439,3.63759,4.45435,3.66356,5.97635,0.69316,3.49651,2.63906,5.54518,3.4012
zeta,1.48345e-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zig,NaN,NaN,NaN,NaN,1.49024e-05,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.1206e-05,NaN,NaN,NaN,NaN,NaN,NaN


#### 4.2 Classifier func given feature table

In [908]:
# If you'll recall, 'Intro to Health Informatics' had by far the smallest number of subtitle words.  (By contrast, ML had like 300x as many words)
# Given that 'Intro to Health Informatics' has the shortest subtitles, we will cap the number of keywords considered to this length of ~1800 words

# len(get_counter_from_list(course_subs_DF[course_subs_DF.course_name == 'Intro to Health Informatics'].subtitles.values, words))

In [909]:
def sorted_limited(s, limit_num=500):
    tmp = s.sort_values(ascending=False)[:limit_num]  # earlier s.order(..)
    tmp.index = range(min(limit_num, len(tmp)))
    return tmp

def classifier(documents, model, stop_words=stopwords_from_data, limit_num=500, transpose_and_sort=True):
    course_posteriors_DF = []
    
    if documents:
        for doc_name, doc_words in documents.items():
            doc_words_nonstopwords = [w for w in doc_words if w not in stop_words]
            model_subset_words = model[model.index.isin(doc_words_nonstopwords)].apply(lambda x: sorted_limited(x, limit_num))
            logPosteriors = model_subset_words.sum(0)
            logPosteriors.rename(doc_name, inplace=True)
            course_posteriors_DF.append(logPosteriors)
        if transpose_and_sort:
            return pd.concat(course_posteriors_DF).transpose().sort_values(ascending=False)
        else:
            return pd.concat(course_posteriors_DF)

In [910]:
tempname = 'Applied Sciences > Medicine and health'
doc_words = list(get_counter_from_list(wiki_page_path_DF[wiki_page_path_DF.page_path_label == tempname].page_text.values, words).keys())
doc_words_nonstopwords = [w for w in doc_words if w not in stopwords_from_data]
# model_subset_words = model[model.index.isin(doc_words_nonstopwords)].apply(lambda x: sorted(x, 10))

In [911]:
model[model.index.isin(doc_words_nonstopwords)]['Advanced Operating Systems'].sort_values(ascending=False)[:100]

global          4.86753
external        4.49981
provided        4.38203
private         4.31749
currently       4.29046
providing        4.2485
unique          4.15888
record          4.14313
active          4.12713
age             4.11087
priority        4.07754
setting         4.06044
namely          4.06044
framework       4.06044
minimum         4.00733
loss            3.93183
ensure          3.93183
interaction     3.91202
reach           3.89182
concerned       3.82864
handle          3.82864
dealing         3.78419
translation     3.73767
community       3.71357
responsible     3.71357
involved        3.71357
failure         3.71357
outside         3.68888
due             3.68888
concurrent      3.68888
                 ...   
establish       3.04452
experiment      3.04452
contain         2.99573
pas             2.99573
central         2.99573
duration        2.99573
expensive       2.99573
guarantee       2.99573
especially      2.99573
particularly    2.94444
pay             

In [912]:
model[model.index.isin(doc_words_nonstopwords)]['Intro to Health Informatics'].sort_values(ascending=False)[:100]

patient           5.55296
clinical          3.58352
record             3.4012
year              3.29584
chronic           3.29584
electronic         3.2581
physician         3.04452
disease           3.04452
hospital          2.99573
medicine          2.94444
fire              2.77259
provider          2.77259
family            2.63906
improve           2.63906
organization      2.63906
recognize         2.48491
setting           2.48491
personal          2.48491
home              2.48491
population         2.3979
provided           2.3979
delivery           2.3979
insurance          2.3979
practice          2.30259
survey            2.30259
ability           2.30259
foundation        2.19723
quality           2.19723
role              2.19723
doctor            2.19723
                   ...   
government        1.38631
growing           1.38631
professional      1.38631
potential         1.38631
central           1.38631
preventive        1.38631
poor              1.38631
increasingly

In [913]:
model = NB_model_features_DF

tempname = 'Applied Sciences > Medicine and health'
tempdict = get_counter_from_list(wiki_page_path_DF[wiki_page_path_DF.page_path_label == tempname].page_text.values, words)

print('{} Matches: \n'.format(tempname), classifier({tempname:list(tempdict.keys())}, model))

Applied Sciences > Medicine and health Matches: 
 course_name
Advanced Operating Systems                                    713.700608
Computer Vision                                               686.065047
Reinforcement Learning                                        652.162579
Machine Learning                                              637.523063
Software Architecture and Design                              631.831520
Introduction to Information Security                          606.114427
Intro to Cyber Physical Systems Security                      567.737348
Introduction to Operating Systems                             560.826785
Knowledge-Based Artificial Intelligence: Cognitive Systems    544.651361
Embedded Systems Optimization                                 504.807835
Software Development Process                                  474.436915
Computational Photography                                     460.233029
Human-Computer Interaction                                    

In [914]:
model = NB_model_features_DF
tempdict = get_counter_from_list(wiki_page_path_DF[wiki_page_path_DF['category_h2_1'] == 'Formal Sciences'].page_text.values, words)

print('Formal Sciences Matches: \n', classifier({'Formal Sciences':list(tempdict.keys())}, model))

Formal Sciences Matches: 
 course_name
Advanced Operating Systems                                    1688.276060
Computer Vision                                               1646.204672
Reinforcement Learning                                        1623.580033
Machine Learning                                              1609.014214
Introduction to Operating Systems                             1501.662847
High Performance Computer Architecture                        1487.206254
Introduction to Information Security                          1484.422550
Software Architecture and Design                              1447.822311
Embedded Systems Optimization                                 1415.334032
Intro to Graduate Algorithms                                  1412.030171
Knowledge-Based Artificial Intelligence: Cognitive Systems    1402.072511
Computational Photography                                     1386.988545
Artificial Intelligence                                       1323.651700

Scary believable :DDDD

#### 4.3 Pre-compute table of P( Course | Interests ) 

In [915]:
work_list = []

model = NB_model_features_DF

print('Pre-computing course rankings for each interest.')
for idx, interest in wiki_page_path_DF.iterrows():
    work_dict = get_counter_from_list(wiki_page_path_DF.iloc[idx].page_text, words)
    print(interest.page_path_label)
    work_list.append(classifier({interest.page_path_label:list(work_dict.keys())}, model))

NB_model_recommendations_DF = pd.concat(work_list, axis=1); #display(NB_model_recommendations_DF.transpose())

Pre-computing course rankings for each interest.
Humanities
Humanities > Arts
Humanities > Arts > Performing arts
Humanities > Arts > Performing arts > Music
Humanities > Arts > Performing arts > Music > Accompanying
Humanities > Arts > Performing arts > Music > Chamber music
Humanities > Arts > Performing arts > Music > Church music
Humanities > Arts > Performing arts > Music > Conducting
Humanities > Arts > Performing arts > Music > Early music
Humanities > Arts > Performing arts > Music > Jazz studies
Humanities > Arts > Performing arts > Music > Musical composition
Humanities > Arts > Performing arts > Music > Music education
Humanities > Arts > Performing arts > Music > Music history
Humanities > Arts > Performing arts > Music > Musicology
Humanities > Arts > Performing arts > Music > Ethnomusicology
Humanities > Arts > Performing arts > Music > Music theory
Humanities > Arts > Performing arts > Music > Orchestral studies
Humanities > Arts > Performing arts > Music > Organology
Hu

/Users/cesleemontgomery/miniconda3/envs/CourseRec/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [916]:
NB_model_recommendations_DF.transpose().tail(20)

,Advanced Operating Systems,Artificial Intelligence,Artificial Intelligence for Robotics,Compilers - Theory and Practice,Computational Photography,Computer Networks,Computer Vision,Data and Visual Analytics,Database Systems Concepts and Design,Educational Technology,...,Introduction to Operating Systems,Knowledge-Based Artificial Intelligence: Cognitive Systems,Machine Learning,Machine Learning for Trading,Network Security,Reinforcement Learning,Software Analysis,Software Architecture and Design,Software Development Process,Special Topics: Big Data for Health Informatics
Applied Sciences > Medicine and health > Radiology,452.872918,255.138615,200.239065,246.748091,350.074367,221.319960,486.934301,219.475454,171.394828,247.165402,...,349.962830,324.963579,398.796623,223.993718,152.638986,389.429878,242.699964,380.212917,276.743956,178.065851
Applied Sciences > Medicine and health > Recreational therapy,187.397596,89.995867,52.530306,77.584460,91.351183,71.363444,141.168799,71.115658,77.734254,122.801346,...,140.744797,127.637457,124.061917,87.285693,51.789416,134.125318,78.864874,154.691235,115.828664,48.002611
Applied Sciences > Medicine and health > Rehabilitation medicine,107.734729,74.095636,41.953578,57.193312,62.235199,43.334683,83.121576,52.510812,40.903472,65.165812,...,91.459108,79.957227,94.755969,40.367575,46.458964,103.421677,48.077400,81.587565,63.986999,70.954709
Applied Sciences > Medicine and health > Respiratory therapy,332.826449,157.350057,128.392324,172.935159,167.046941,160.760404,263.830992,128.718600,130.579593,232.740823,...,297.150521,239.626752,252.480946,132.003283,120.279165,285.562058,156.754358,301.947392,212.012313,131.529869
Applied Sciences > Medicine and health > Sleep medicine,366.669832,200.549727,130.830209,205.904212,223.728680,171.345557,332.027597,159.732004,140.270335,211.996476,...,288.769150,297.836922,314.403198,177.266715,141.632802,318.996697,187.849559,369.240693,241.967816,140.797632
Applied Sciences > Medicine and health > Speech-language pathology,342.941260,168.075596,124.772565,176.100691,165.403806,144.492375,252.285396,151.734088,136.420089,243.662496,...,264.007474,244.441867,251.643193,143.129026,114.544662,271.741329,140.005337,299.781205,218.234708,138.210046
Applied Sciences > Medicine and health > Sports medicine,138.823245,73.960425,42.779396,61.226999,82.632973,69.467205,112.644191,55.343205,68.352159,102.880243,...,98.309475,100.378230,106.578570,62.857679,47.879480,103.335373,56.743275,122.152344,92.296087,80.143987
Applied Sciences > Medicine and health > Surgery,582.050449,323.510611,239.474875,368.257172,402.000768,285.677958,584.489722,290.299876,218.632522,319.572523,...,457.154556,456.511824,505.656591,266.856135,212.520628,487.989629,282.778972,524.033218,372.355326,249.090011
Applied Sciences > Medicine and health > Surgery > Bariatric surgery,378.409453,242.043583,178.253842,212.092732,233.576029,223.013790,369.307501,182.459125,130.910337,216.926080,...,330.824763,300.506883,344.131950,197.016371,141.065347,352.829727,207.594635,322.355075,217.039706,169.600792
Applied Sciences > Medicine and health > Surgery > Cardiothoracic surgery,311.893266,242.667967,160.100018,184.421805,207.421017,174.848165,341.095818,161.127286,134.094369,186.263714,...,278.501126,229.689030,295.273462,149.330376,105.849322,311.071936,163.164592,278.042998,204.416871,143.748540


#### 4.4 Extract Explanations

### 5 Offline Evaluation (via manually generated scenarios)

This is the closest we'll come to evaluation since we don't have historical user data.  Let's compare manually to other sane baselines.

In [917]:
course_key = {
    #Hand-picked as more related to social/humanities disciplines
    'Human-Computer Interaction': 'red',
    'Intro to Health Informatics': 'red',
    'Educational Technology': 'red',

    #Hand-picked as more related to engineering/systems disciplines
    'Database Systems Concepts and Design': 'green',
    'Computer Networks': 'green',
    'Compilers - Theory and Practice': 'green',

    #Hand-picked as more related to math/highly quantitative disciplines
    'Machine Learning': 'blue',
    'Reinforcement Learning': 'blue',
    'Data and Visual Analytics': 'blue'
}

def color_hand_classified_courses(course, key=course_key):
    """
    Takes a course (string) and colors specific courses 
    according to a hand-created classification key, designed
    to compare broad sub-categories of Computer Science 
    across academic interests.
    """
    if course in key.keys():
        color = key[course]
    else: color = 'gainsboro'
    return 'background-color: %s' % color

In [918]:
#1. write Rank x Categories (values=Course)
def build_comparison_DF(df): 
    work_df_ranked = pd.DataFrame(index=pd.Index(range(1,29), name='rank'))
    work_df = df
    for interest in work_df:
        tempdf = work_df[interest].rank(ascending=False).astype('int').rename('rank').reset_index().set_index('rank')
    #     print(work_df[interest].rank(ascending=False).sort_values())
    #     break
        tempdf.columns = [interest]
        work_df_ranked = work_df_ranked.join(tempdf)
    return work_df_ranked

#### 5.1 Categories Scenarios (w/Explanations) (Comparison to Random or TF-IDF)

In [919]:
#1. write Rank x Categories (values=Course) ; get categories (will serve as spectrum from social to mathy/engr-y [])
categories_DF = pd.DataFrame(NB_model_recommendations_DF.loc[:,wiki_page_path_DF.category_h2_1.unique()])

#2. Declare course key, scan table for course key and color with course.color
(build_comparison_DF(categories_DF).style
  .applymap(color_hand_classified_courses)
  .set_caption('Course Rank by Academic Discipline'))

,Humanities,Social sciences,Natural Sciences,Formal Sciences,Applied Sciences
rank,,,,,
1,Advanced Operating Systems,Advanced Operating Systems,Advanced Operating Systems,Advanced Operating Systems,Advanced Operating Systems
2,Computer Vision,Computer Vision,Computer Vision,Computer Vision,Intro to Cyber Physical Systems Security
3,Reinforcement Learning,Reinforcement Learning,Machine Learning,Machine Learning,Computer Vision
4,Software Architecture and Design,Software Architecture and Design,Reinforcement Learning,Reinforcement Learning,Software Architecture and Design
5,Machine Learning,Machine Learning,Software Architecture and Design,Software Architecture and Design,Introduction to Information Security
6,Introduction to Information Security,Introduction to Information Security,Knowledge-Based Artificial Intelligence: Cognitive Systems,Introduction to Operating Systems,Reinforcement Learning
7,Knowledge-Based Artificial Intelligence: Cognitive Systems,Knowledge-Based Artificial Intelligence: Cognitive Systems,Introduction to Information Security,Introduction to Information Security,Machine Learning
8,Introduction to Operating Systems,Intro to Cyber Physical Systems Security,Introduction to Operating Systems,Knowledge-Based Artificial Intelligence: Cognitive Systems,Knowledge-Based Artificial Intelligence: Cognitive Systems
9,Computational Photography,Introduction to Operating Systems,Intro to Cyber Physical Systems Security,Embedded Systems Optimization,Introduction to Operating Systems


#### 5.2 Computer Science vs Computer Science classes

In [920]:
computer_science_interests_courses_keys = {
    'Formal Sciences > Computer Science > Logic in computer science':[],
       'Formal Sciences > Computer Science > Algorithms':['Intro to Graduate Algorithms'],
       'Formal Sciences > Computer Science > Artificial intelligence':['Artificial Intelligence', 'Artificial Intelligence for Robotics', 'Machine Learning', 'Knowledge-Based Artificial Intelligence: Cognitive Systems', 'Computer Vision', 'Computational Photography', 'Machine Learning for Trading', 'Reinforcement Learning'],
       'Formal Sciences > Computer Science > Data structures':['Database Systems Concepts and Design', 'Data and Visual Analytics'],
       'Formal Sciences > Computer Science > Computer architecture':['Intro to High-Performance Computing', 'High Performance Computer Architecture'],
       'Formal Sciences > Computer Science > Computer graphics':['Intro to High-Performance Computing', 'High Performance Computer Architecture'],
       'Formal Sciences > Computer Science > Computer communications (networks)':['Computer Networks', 'Network Security', 'Intro to Cyber Physical Systems Security'],
       'Formal Sciences > Computer Science > Computer security and reliability':['Network Security', 'Introduction to Information Security'],
       'Formal Sciences > Computer Science > Computing in mathematics, natural sciences, engineering, and medicine':['Data and Visual Analytics', 'Machine Learning', 'Intro to Health Informatics', 'Special Topics: Big Data for Health Informatics', 'Computational Photography'],
       'Formal Sciences > Computer Science > Computing in social sciences, arts, humanities, and professions':['Data and Visual Analytics', 'Educational Technology', 'Machine Learning for Trading'],
       'Formal Sciences > Computer Science > Distributed computing':['Intro to High-Performance Computing', 'High Performance Computer Architecture'],
       'Formal Sciences > Computer Science > Human-computer interaction':['Human-Computer Interaction'],
       'Formal Sciences > Computer Science > Operating systems':['Introduction to Operating Systems', 'Advanced Operating Systems', 'Embedded Systems Optimization'],
       'Formal Sciences > Computer Science > Parallel computing':['Intro to High-Performance Computing', 'High Performance Computer Architecture'],
       'Formal Sciences > Computer Science > Programming languages':['Compilers - Theory and Practice'],
       'Formal Sciences > Computer Science > Quantum computing':['Intro to High-Performance Computing', 'High Performance Computer Architecture'],
       'Formal Sciences > Computer Science > Software engineering':['Database Systems Concepts and Design', 'Software Analysis', 'Software Development Process', 'Software Architecture and Design'],
       'Formal Sciences > Computer Science > Theory of computation':['Intro to Graduate Algorithms'],
       'Formal Sciences > Computer Science > VLSI design':[]
}

def color_course_match(course):
    if course:
        color = 'blue'
    else: color = 'gainsboro'
    return 'background-color: %s' % color

def color_hand_classified_CS_course(series, course_key=computer_science_interests_courses_keys):
    """
    Takes a course and colors specific courses 
    according to a hand-created classification key, designed
    to compare broad sub-categories of Computer Science 
    across academic interests.
    """
    return series.apply(lambda course: color_course_match(course in course_key[series.name]))


In [921]:
# #1. write Rank x Categories (values=Course) ; get computer science courses
tempdf = (wiki_page_path_DF.loc[(wiki_page_path_DF['sub_category_h3_2'] == 'Computer Science') 
                                    & (pd.notnull(wiki_page_path_DF['topic_3']))
                                    & (pd.isnull(wiki_page_path_DF['subtopic_4']))
                                    & (pd.isnull(wiki_page_path_DF['subtopic_5'])), ['page_path_label', 'sub_category_h3_2']])
tempdf = NB_model_recommendations_DF.transpose().join(tempdf.set_index('page_path_label'), how='inner')

temp_df = (build_comparison_DF(tempdf.iloc[:, :-1].transpose()).style
  .apply(color_hand_classified_CS_course, axis=0))

html = temp_df.render()
from IPython.display import display, HTML
display( HTML( html ) )
# import imgkit
# imgkit.from_string(html, 'v1Offline evaluationCS.png')

,Formal Sciences > Computer Science > Logic in computer science,Formal Sciences > Computer Science > Algorithms,Formal Sciences > Computer Science > Artificial intelligence,Formal Sciences > Computer Science > Data structures,Formal Sciences > Computer Science > Computer architecture,Formal Sciences > Computer Science > Computer graphics,Formal Sciences > Computer Science > Computer communications (networks),Formal Sciences > Computer Science > Computer security and reliability,"Formal Sciences > Computer Science > Computing in mathematics, natural sciences, engineering, and medicine","Formal Sciences > Computer Science > Computing in social sciences, arts, humanities, and professions",Formal Sciences > Computer Science > Distributed computing,Formal Sciences > Computer Science > Human-computer interaction,Formal Sciences > Computer Science > Operating systems,Formal Sciences > Computer Science > Parallel computing,Formal Sciences > Computer Science > Programming languages,Formal Sciences > Computer Science > Quantum computing,Formal Sciences > Computer Science > Software engineering,Formal Sciences > Computer Science > Theory of computation,Formal Sciences > Computer Science > VLSI design
rank,,,,,,,,,,,,,,,,,,,
1,Advanced Operating Systems,Machine Learning,Machine Learning,Advanced Operating Systems,Advanced Operating Systems,Computer Vision,Advanced Operating Systems,Advanced Operating Systems,Advanced Operating Systems,Advanced Operating Systems,Advanced Operating Systems,Advanced Operating Systems,Advanced Operating Systems,Advanced Operating Systems,Advanced Operating Systems,Advanced Operating Systems,Advanced Operating Systems,Reinforcement Learning,Advanced Operating Systems
2,Computer Vision,Reinforcement Learning,Reinforcement Learning,Introduction to Operating Systems,Software Architecture and Design,Advanced Operating Systems,Introduction to Information Security,Introduction to Information Security,Computer Vision,Computer Vision,Introduction to Operating Systems,Computer Vision,Introduction to Operating Systems,Introduction to Operating Systems,Software Architecture and Design,Computer Vision,Software Architecture and Design,Machine Learning,Computer Vision
3,Machine Learning,Advanced Operating Systems,Advanced Operating Systems,Software Architecture and Design,Computer Vision,Machine Learning,Intro to Cyber Physical Systems Security,Introduction to Operating Systems,Reinforcement Learning,Reinforcement Learning,Software Architecture and Design,Software Architecture and Design,Introduction to Information Security,Software Architecture and Design,Introduction to Operating Systems,Machine Learning,Reinforcement Learning,Advanced Operating Systems,Software Architecture and Design
4,Knowledge-Based Artificial Intelligence: Cognitive Systems,Computer Vision,Computer Vision,Embedded Systems Optimization,High Performance Computer Architecture,Reinforcement Learning,Introduction to Operating Systems,Intro to Cyber Physical Systems Security,Machine Learning,Software Architecture and Design,Reinforcement Learning,Introduction to Operating Systems,Software Architecture and Design,Embedded Systems Optimization,Computer Vision,Reinforcement Learning,Introduction to Information Security,Software Architecture and Design,Introduction to Operating Systems
5,Reinforcement Learning,Software Architecture and Design,Software Architecture and Design,Introduction to Information Security,Introduction to Operating Systems,Software Architecture and Design,Software Architecture and Design,Software Architecture and Design,Software Architecture and Design,Machine Learning,Computer Vision,Machine Learning,Intro to Cyber Physical Systems Security,High Performance Computer Architecture,Machine Learning,Introduction to Information Security,Machine Learning,Computer Vision,High Performance Computer Architecture
6,Software Architecture and Design,Introduction to Operating Systems,Introduction to Information Security,Computer Vision,Machine Learnin

Does seem somewhat better, atleast for some categories.  Still not great on its own.  Definitely should combine this with more opinionated syllabus model.  Only other possibility for improvement is somehow normalizing for length in calculation.

TODO - Actually it occurs to me that P(course | interests) could be a ratio of matches found / matches not found.  Maybe this will reduce the dominance of particularly wordy.  Also could try named entity recognition to get at more valuable keywords.

Regardless, I will proceed with syllabus model and hope for more improvement with that.

### 6. Write to Firebase

In [922]:
NB_model_recommendations_DF.transpose().head()

,Advanced Operating Systems,Artificial Intelligence,Artificial Intelligence for Robotics,Compilers - Theory and Practice,Computational Photography,Computer Networks,Computer Vision,Data and Visual Analytics,Database Systems Concepts and Design,Educational Technology,...,Introduction to Operating Systems,Knowledge-Based Artificial Intelligence: Cognitive Systems,Machine Learning,Machine Learning for Trading,Network Security,Reinforcement Learning,Software Analysis,Software Architecture and Design,Software Development Process,Special Topics: Big Data for Health Informatics
Humanities,612.087749,334.675286,241.191817,331.604000,448.260501,231.991568,599.551135,278.882254,222.628887,381.285685,...,459.151505,468.256613,539.078233,278.835802,197.336818,589.782248,311.955616,541.316688,424.355064,191.838326
Humanities > Arts,339.996604,164.259766,127.537894,179.914329,267.735235,143.737402,377.554678,152.463602,127.853759,177.203021,...,235.328471,296.046856,314.843377,140.253862,86.062081,314.112939,156.605193,347.576371,240.663876,114.026183
Humanities > Arts > Performing arts,372.830859,235.811761,165.534905,200.123254,298.213839,160.520457,392.355005,154.863067,135.665290,228.666673,...,273.148131,281.646353,338.908632,173.019487,109.848069,357.252227,178.572856,329.224664,242.407330,107.624890
Humanities > Arts > Performing arts > Music,1022.007458,627.443049,470.216147,529.187674,695.260809,444.131683,1055.158796,483.617645,377.149454,577.135306,...,761.912871,766.363085,1008.149155,481.270863,310.284722,996.379484,467.244883,938.376992,635.004486,328.994707
Humanities > Arts > Performing arts > Music > Accompanying,89.020178,75.882885,35.045173,38.040705,53.995011,35.562096,84.960485,40.659725,33.811172,42.729610,...,65.693068,61.687778,92.982127,46.050020,30.237804,95.460671,52.071892,85.772041,54.632963,30.090516


In [923]:
# UI_interests = OrderedDict({
#     'Computer Science': {'Formal Sciences':['Computer Science']},
#     'Math & Statistics': {'Formal Sciences':['Mathematics', 'Statistics']},
#     'Engineering & Tech': {'Applied Sciences':['Engineering and technology']}, 
#     'Business': 'Business',
#     'Medicine & Health': {'Applied Sciences':['Medicine and health']},
#     'Natural Sciences': 'Natural Sciences',
#     'Social Sciences': 'Social sciences',
#     'Humanities': 'Humanities'
# })

# worklist = []
# # for i, (key, value) in enumerate(d.items())
# for i, (interest_label, interest_id) in enumerate(UI_interests.items()):
#     if isinstance(interest_id, dict):
#         category_id = list(interest_id.values())[0]
#         tempdf = pd.DataFrame(wiki_page_path_DF.loc[(wiki_page_path_DF.sub_category_h3_2.isin(list(interest_id.values())[0])), 'page_path_label'])
#         course_recommendations = pd.merge(tempdf, NB_model_recommendations_DF.transpose(), left_on='page_path_label', right_index=True).drop(columns=['page_path_label']).mean().to_dict()
#     elif interest_label == 'Business':
#         category_id = [interest_id]
#         tempdf = pd.DataFrame(wiki_page_path_DF.loc[(wiki_page_path_DF.sub_category_h3_2 == interest_id), 'page_path_label'])
#         course_recommendations = pd.merge(tempdf, NB_model_recommendations_DF.transpose(), left_on='page_path_label', right_index=True).drop(columns=['page_path_label']).to_dict('records')[0]
#     else:
#         category_id = [interest_id]
#         course_recommendations = NB_model_recommendations_DF.transpose().loc[category_id[0]].to_dict()
#     #categoryLabel, categoryIDs, course_recommendations:{category:prediction}
#     worklist.append([interest_label, category_id, course_recommendations])
    
# workdf = pd.DataFrame(worklist, columns=['interest_label', 'category_id', 'course_recommendations'])
# workdf

In [924]:
# # Some help from: https://medium.com/@hmurari/cloud-firestore-batch-transactions-how-to-migrate-a-large-amounts-of-data-336e61efbe7c
# # Use the application default credentials
# cred = credentials.ApplicationDefault()
# firebase_admin.initialize_app(cred, {
#   'projectId': 'courserec-adfe3',
# })

# db = firestore.client()

In [925]:
# recommendations_collection = db.collection(u'recommendations')

# #for batching
# total = len(workdf)
# counter = 0

# # Start a batch
# batch = db.batch()
# for idx, record in workdf.iterrows():
#     # Commit the batch at every 500th record.
#     if counter % 500 == 0:
#         if counter > 0:
#             print('Committing..')
#             batch.commit()

#         # Start a new batch for the next iteration.
#         batch = db.batch()
#     counter += 1
#     print(str(counter) + str('/') + str(total) + ': ' + str(record.interest_label))
#     record_id = record.interest_label
#     record_ref = recommendations_collection.document(str(record_id))
#     # Include current record in batch
#     batch.set(record_ref, 
#         {
#             "categoryLabel": record.interest_label,
#             "courseRecommendations": record.course_recommendations
#     })

# # Include current record in batch
# if counter % 500 != 0:
#     print('Committing..')
#     batch.commit()

### 7. Future Directions
- Add superset categories to documents bag of words to increase coverage?
- Turn into model 
- Limit unrelated interests
- Use MMR to evaluate (can normalize)

In [926]:
# class MN_NaiveBayes:

#     """
#     Constructor for MN_NaiveBayes.
#     Initializes overall counts of positive, negative, and neutral classes.
#     Initializes overall document count for use in a priori class probability
#     calculation.
#     Initializes pos, neg, and neutral feature count dictionaries.
#     """
#     def __init__(self, pos, neg):

#     """
#     An implementation of Jurafsky's MN Bayes Network
#     algorithm.
#     """
#     def train(self):